In [19]:
#select intepreter and change environment to anaconda
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib
import xmltodict
from xml.dom import minidom
import time


#handles command line input parameter
import sys
import os

# testing file is from 10,000 reads, so mapping percentage is expected to be out of 10,000
# blast_tsv_file = sys.argv[1]
# blast_tsv_file2 = sys.argv[2]
# piePath = sys.argv[3]
blast_tsv_file = 'LARP6.CTRL_IN1.umi.r1.fqdown_unmappedblast_downsampled_blastn.tsv'
blast_tsv_file2 = 'LARP6.CTRL_IN1.umi.r1.fqdown_unmappedblast_downsampled_blastn.tsv'
piePath = 'combined.png'
piePathSample = os.path.splitext(piePath)[0]
piePathSample2 = os.path.split(piePathSample)
pieName = piePathSample2[1]


#ncbi query functions
def esearch(term, db='gds'):
    """
    Queries NCBI using the esearch utility. GEO ('gds') database is used as default for search term.
    """
    url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db={db}&term={term}&retmax=5000&usehistory=y'
    response = urllib.request.urlopen(url)
    return response.read()


def get_esummary(esearch_string, db='gds'):
    """
    Parses a http response in XML format to obtain the webenv and querykey tokens.
    Uses NCBI eutils to transform these tokens into web summaries of GEO (db='gds') datasets.
    """
    xmldoc = minidom.parseString(esearch_string)
    try:
        webenv = xmldoc.getElementsByTagName('WebEnv')[0].firstChild.data
        querykey = xmldoc.getElementsByTagName('QueryKey')[0].firstChild.data
        host = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi'
        params = f'?db={db}&version=2.0&query_key={querykey}&WebEnv={webenv}'
        url = host + params
        response = urllib.request.urlopen(url)
        return response.read()
    except IndexError as e:
        print(f"Unparsable publication string ({e}, search={esearch_string}")
        return ""

#matplotlib to build the piechart
fig, ((ax1, ax2, ax5), (ax3, ax4, ax6)) = plt.subplots(2, 3, figsize=(9, 5))
fig.tight_layout(h_pad=2)
plt.subplots_adjust(bottom=0, right=1.5, top=1.5)
ax1.set_title('blastn piechart')
ax2.set_title('blastn other')
ax3.set_title('blastx piechart')
ax4.set_title('blastx other')
ax5.set_title('blastn percentage')
ax6.set_title('blastx percentage')





# Read the blast n file
blast_N_tsv_Input = pd.read_csv(blast_tsv_file, header=None, sep='\t')
blast_N_tsv_Input.columns = ['qseqid','sseqid','pident','length','mismatch','gapopen','qstart','qend','sstart','send','evalue','bitscore']

df2 = blast_N_tsv_Input[['qseqid','sseqid','evalue']].copy()

# Read the blast x file
blast_X_tsv_Input = pd.read_csv(blast_tsv_file2, header=None, sep='\t')
blast_X_tsv_Input.columns = ['qseqid','sseqid','pident','length','mismatch','gapopen','qstart','qend','sstart','send','evalue','bitscore']




### still need to reorganize the code for each different file below.


#blast n output default is by best hit/lowest e value score, therefore add the first sseqid result for each qseqid
# Get the unique sseqid and the corresponding qseqid, so we know which sequence is mapped by blasted.

blastn_searchUnique_sseqid = []
blastn_qseqid_found = []
for idx in blast_N_tsv_Input.index:
        if blast_N_tsv_Input['qseqid'][idx] not in blastn_qseqid_found:
                blastn_qseqid_found.append(blast_N_tsv_Input['qseqid'][idx])
                blastn_searchUnique_sseqid.append(blast_N_tsv_Input['sseqid'][idx])
blastn_unique_sseqid = np.array(blastn_searchUnique_sseqid)

#blast x output default is by best hit/lowest e value score, therefore add the first sseqid result for each qseqid
# Get the unique sseqid and the corresponding qseqid, so we know which sequence is mapped by blasted.
blastx_searchUnique_sseqid = []
blastx_qseqid_found = []
for idx in blast_X_tsv_Input.index:
        if blast_X_tsv_Input['qseqid'][idx] not in blastx_qseqid_found:
                blastx_qseqid_found.append(blast_X_tsv_Input['qseqid'][idx])
blastx_unique_sseqid = np.array(blastx_searchUnique_sseqid)

#Make a dataframe blast_n_sseqid_df for blast n sseqid and record the size of the mapped in blast_n_mapped_size
blast_n_sseqid_df = pd.DataFrame(data = blastn_unique_sseqid, columns=['sseqid'])
print(blast_n_sseqid_df)
blast_n_mapped_size = blast_n_sseqid_df.size

#Make a dataframe blast_x_sseqid_df for blast x sseqid and record the size of the mapped in blast_x_mapped_size
blast_x_sseqid_df = pd.DataFrame(data = blastx_unique_sseqid, columns=['sseqid'])
print(blast_x_sseqid_df)
blast_x_mapped_size = blast_x_sseqid_df.size

# Blast n
sseq_count_series1 = blast_n_sseqid_df['sseqid'].value_counts()
print(sseq_count_series1)

# Blast x
sseq_count_series4 = blast_x_sseqid_df['sseqid'].value_counts()
print(sseq_count_series4)

#loop through series to determine which elements to remove and add into "other" column
# Blast n
blast_n_genes_notTop_hit = []
blast_n_otherCategory_size = 0
blast_n_other_store = {}
# Blast x
blast_x_genes_notTop_hit = []
blast_x_otherCategory_size = 0
blast_x_other_store = {}

#blast n number mapped
print(blast_n_mapped_size)

#blast x number mapped
print(blast_x_mapped_size)

# filtering out the sequence's top three mapped genes counts for blast n
only_2_n = 2
for index,values in sseq_count_series1.iteritems():
        if(values < (0.001)*(blast_n_mapped_size)) or only_2_n == 0:
                blast_n_genes_notTop_hit.append(index)
                blast_n_otherCategory_size += values
                blast_n_other_store[values] = index
        else:
            only_2_n -= 1

# filtering out the sequence's top three mapped genes counts for blast x
only_2_x = 2
for index,values in sseq_count_series4.iteritems():
        if(values < (0.001)*(blast_x_mapped_size)) or only_2_x == 0:
                blast_x_genes_notTop_hit.append(index)
                blast_x_otherCategory_size += values
                blast_x_other_store[values] = index
        else:
            only_2_x -= 1

# take the top three genes in the other category for blast n
top_3_in_blastn_other = []
increment1 = 0
for key,value in sorted(blast_n_other_store.items(),reverse=True):
        if increment1 <= 2:
                top_3_in_blastn_other.append((key,value))
                increment1+=1
        else:
             break              

# take the top three genes in the other category for blast x
top_3_in_blastx_other = []
increment2 = 0
for key,value in sorted(blast_x_other_store.items(),reverse=True):
        if increment2 <= 2:
                top_3_in_blastx_other.append((key,value))
                increment2+=1
        else:
             break

# Store and calculate the ratio of the three selected genes blast n in the other category and plot them onto bar chart
blastn_bar_value_count = []
blastn_bar_ratios = []
blastn_bar_labels = []  
blastn_bar_untranslated_labels = []  

for idx in range(len(top_3_in_blastn_other)):
        blastn_bar_value_count.append(top_3_in_blastn_other[idx][0])
        blastn_bar_ratios.append(top_3_in_blastn_other[idx][0]/blast_n_otherCategory_size)
        blastn_bar_untranslated_labels.append(top_3_in_blastn_other[idx][1])

#replace index sseqids with ncbi name, for blast n bar chart
for idx in range(len(blastn_bar_untranslated_labels)):
        term = str(blastn_bar_untranslated_labels[idx])
        print(term)
        esearch_string = esearch(term=term, db='nucleotide')
        time.sleep(0.1)
        result = get_esummary(esearch_string=esearch_string, db='nucleotide')
        result = xmltodict.parse(result)
        sseq_name = result['eSummaryResult']['DocumentSummarySet']['DocumentSummary']['Title']
        blastn_bar_labels.append(sseq_name)                          

        
# Store and calculate the ratio of the three selected genes blast x in the other category and plot them onto bar chart
blastx_bar_value_count = []   
blastx_bar_ratio = []
blastx_bar_labels = []
blastx_bar_untranslated_labels = []

for idx in range(len(top_3_in_blastx_other)):
        blastx_bar_ratio.append(top_3_in_blastx_other[idx][0]/blast_x_otherCategory_size)
        blastx_bar_untranslated_labels.append(top_3_in_blastx_other[idx][1])
        blastx_bar_value_count.append(top_3_in_blastx_other[idx][0])

#replace index sseqids with ncbi name, for blast x bar chart
for idx in range(len(blastx_bar_untranslated_labels)):
        term = str(blastx_bar_untranslated_labels[idx])
        print(term)
        esearch_string = esearch(term=term, db='nucleotide')
        time.sleep(0.3)
        result = get_esummary(esearch_string=esearch_string, db='nucleotide')
        result = xmltodict.parse(result)
        sseq_name = result['eSummaryResult']['DocumentSummarySet']['DocumentSummary']['Title']
        blastx_bar_labels.append(sseq_name) 
                         

#remove the low counts genes from the blast n
sseq_count_series2 = sseq_count_series1.drop(blast_n_genes_notTop_hit)
#remove the low counts genes from the blast x
sseq_count_series5 = sseq_count_series4.drop(blast_x_genes_notTop_hit)


# Get the gene names for the first pie chart blast n
sseq_name_list = []
sseqid = []
#replace index sseqids with ncbi name
for index,values in sseq_count_series2.iteritems():
        sseqid.append(index)
        term = str(index)
        esearch_string = esearch(term=term, db='nucleotide')
        time.sleep(0.1)
        result = get_esummary(esearch_string=esearch_string, db='nucleotide')
        result = xmltodict.parse(result)
        sseq_name = result['eSummaryResult']['DocumentSummarySet']['DocumentSummary']['Title']
        sseq_name_list.append(sseq_name)
        #sseq_count_series2.rename(index={index:sseq_name})
#replace the sseqids with ncbi query names
replacements = {sseqid:sseq_name_list for sseqid, sseq_name_list in zip(sseqid, sseq_name_list)}
sseq_count_series3 = sseq_count_series2.rename(replacements)
#sseq_count_series2 = sseq_count_series2.rename(index=dict(zip(sseq_name_list,sseqid)))

#generate new pandas series with new element to concatenate with old series
d = {'Other':blast_n_otherCategory_size}
ser = pd.Series(data=d, index=['Other'])

#append new element
sseq_count = sseq_count_series3.append(ser)
#print(sseq_count)

#value_counts returns a pandas series so convert to a data frame
sseq_count_df = pd.DataFrame({'sseqid':sseq_count.index, 'count':sseq_count.values})
print(sseq_count_df)

count = sseq_count_df['count']
sseq = sseq_count_df['sseqid']


# Get the gene names for the first pie chart blast x
sseq_name_list2 = []
sseqid2 = []
#replace index sseqids with ncbi name
for index,values in sseq_count_series5.iteritems():
        sseqid2.append(index)
        term = str(index)
        esearch_string = esearch(term=term, db='nucleotide')
        time.sleep(0.1)
        result = get_esummary(esearch_string=esearch_string, db='nucleotide')
        result = xmltodict.parse(result)
        sseq_name = result['eSummaryResult']['DocumentSummarySet']['DocumentSummary']['Title']
        sseq_name_list2.append(sseq_name)
        #sseq_count_series2.rename(index={index:sseq_name})
#replace the sseqids with ncbi query names
replacements2 = {sseqid2:sseq_name_list2 for sseqid2, sseq_name_list2 in zip(sseqid2, sseq_name_list2)}
sseq_count_series6 = sseq_count_series5.rename(replacements2)
#sseq_count_series2 = sseq_count_series2.rename(index=dict(zip(sseq_name_list,sseqid)))

#generate new pandas series with new element to concatenate with old series
d2 = {'Other':blast_x_otherCategory_size}
ser2 = pd.Series(data=d2, index=['Other'])

#append new element
sseq_count2 = sseq_count_series6.append(ser2)
#print(sseq_count)

#value_counts returns a pandas series so convert to a data frame
sseq_count_df2 = pd.DataFrame({'sseqid':sseq_count2.index, 'count':sseq_count2.values})
print(sseq_count_df2)

count2 = sseq_count_df2['count']
sseq2 = sseq_count_df2['sseqid']



# Parsing in the gene labels and counts data to build the plots.
print(blastn_bar_value_count)
print(blastx_bar_value_count)
# Blast n first pie chart and bar chart
bottom = 1
width = 0.2
wedges1, *_ = ax1.pie(count, labels=[f'{sseq[i]}: {count[i]} ({count[i]/sum(count)*100:.1f}% )' for i in range(len(sseq))], colors=None,autopct='',startangle=45,
        wedgeprops={"linewidth": 1, "edgecolor": "white"})
for j, (height, label, num) in enumerate(reversed([*zip(blastn_bar_ratios, blastn_bar_labels, blastn_bar_value_count)])):
    bottom -= height
    bc = ax2.bar(0, height, width, bottom=bottom, color='C0', label=label,
                 alpha=0.1 + 0.25 * j)
    ax2.bar_label(bc, labels=[f"{height:.0%}({num})"], label_type='center')
ax2.axis('off')
ax2.set_xlim(- 2.5 * 2, 2.5 * 2)
legend = ax2.legend(loc='upper center', bbox_to_anchor=(0.5, 0.02), ncol=1)


# Blast x first pie chart and bar chart
bottom = 1
width = 0.2
wedges2, *_ = ax3.pie(count2, labels=[f'{sseq2[i]}: {count2[i]} ({count2[i]/sum(count2)*100:.1f}% )' for i in range(len(sseq2))], colors=None,autopct='',startangle=45,
        wedgeprops={"linewidth": 1, "edgecolor": "white"})
for j, (height, label, num) in enumerate(reversed([*zip(blastx_bar_ratio, blastx_bar_labels, blastx_bar_value_count)])):
    bottom -= height
    bc = ax4.bar(0, height, width, bottom=bottom, color='C0', label=label,
                 alpha=0.1 + 0.25 * j)
    ax4.bar_label(bc, labels=[f"{height:.0%}({num})"], label_type='center')
ax4.axis('off')
ax4.set_xlim(- 2.5 * 2, 2.5 * 2)
legend2 = ax4.legend(loc='upper center', bbox_to_anchor=(0.5, 0.02), ncol=1)


### Get the percentage of the the blast n/x mapped results
# for ax5
ax5_percentages = [10000 - blast_n_mapped_size, blast_n_mapped_size]
ax5_labels = ['Unmapped', 'Mapped']
ax5.pie(ax5_percentages, labels=[f'{ax5_labels[i]}: {ax5_percentages[i]} ({ax5_percentages[i]/sum(ax5_percentages)*100:.1f}% )' for i in range(len(ax5_labels))], autopct='',
       colors=['skyblue', 'gray'], labeldistance=1.1)

# for ax6
ax6_percentages = [10000 - blast_x_mapped_size, blast_x_mapped_size]
ax6_labels = ['Unmapped', 'Mapped']
ax6.pie(ax6_percentages, labels=[f'{ax6_labels[i]}: {ax6_percentages[i]} ({ax6_percentages[i]/sum(ax6_percentages)*100:.1f}% )' for i in range(len(ax6_labels))], autopct='',
       colors=['skyblue', 'gray'], labeldistance=1.1)

# Adding Circle in Pie chart
circle1 = plt.Circle((0, 0), radius=0.6, color='white')
ax5.add_patch(circle1)
circle2 = plt.Circle((0, 0), radius=0.6, color='white')
ax6.add_patch(circle2)

text1 = str(blast_n_mapped_size) + " out of 10000 sequences mapped"
text2 = str(blast_x_mapped_size) + " out of 10000 sequences mapped"

# plt.title('Blastn and Blastx Unmapped Sequences Summary ' + pieName)
ax5.text(0.95, 2.2, text1, transform=ax6.transAxes, fontsize=14,
        verticalalignment='top', bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=1'))
ax6.text(0.95, 0.95, text2, transform=ax6.transAxes, fontsize=14,
        verticalalignment='top', bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=1'))

fig.tight_layout(pad=5.0)
plt.show(block=True)


              sseqid
0         OV277454.1
1         MN894805.1
2         CP076413.1
3     XR_006695749.1
4         OV277454.1
...              ...
9331     NR_103099.1
9332  XR_006269445.1
9333      KF104644.1
9334      CP068261.2
9335      AL590668.5

[9336 rows x 1 columns]
Empty DataFrame
Columns: [sseqid]
Index: []
OV277454.1        5223
MN894805.1         586
XR_006695749.1     152
OV277463.1         151
OV277457.1         108
                  ... 
OV179167.1           1
HQ659117.1           1
AC254813.1           1
CP087586.1           1
AL590668.5           1
Name: sseqid, Length: 1290, dtype: int64
Series([], Name: sseqid, dtype: int64)
9336
0
XR_006695749.1
OV277463.1
OV277457.1


/tmp/ipykernel_34618/640869443.py:256: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sseq_count = sseq_count_series3.append(ser)
/tmp/ipykernel_34618/640869443.py:291: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sseq_count2 = sseq_count_series6.append(ser2)
/tmp/ipykernel_34618/640869443.py:324: RuntimeWarning: invalid value encountered in long_scalars
  wedges2, *_ = ax3.pie(count2, labels=[f'{sseq2[i]}: {count2[i]} ({count2[i]/sum(count2)*100:.1f}% )' for i in range(len(sseq2))], colors=None,autopct='',startangle=45,
/opt/conda/lib/python3.9/site-packages/matplotlib/axes/_axes.py:3050: RuntimeWarning: invalid value encountered in true_divide
  x = x / sx
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argume

                                              sseqid  count
0        Meles meles genome assembly, chromosome: 13   5223
1  Homo sapiens haplogroup H3v+16093 mitochondrio...    586
2                                              Other   3527
  sseqid  count
0  Other      0
[152, 151, 108]
[]


ValueError: need at least one array to concatenate

posx and posy should be finite values


ValueError: need at least one array to concatenate

<Figure size 648x360 with 6 Axes>